In [1]:
import sqlite3
import pandas as pd
import numpy as np
import sys
import glob, os
from pyteomics import mgf

In [11]:
BASE_MAXQUANT_DIR = '/Users/darylwilding-mcbride/Downloads/experiments/YHE-R/YHE114_01_YHE211_01-combined'
MAXQUANT_TXT_DIR = '{}/txt'.format(BASE_MAXQUANT_DIR)
ALLPEPTIDES_FILENAME = '{}/allPeptides.txt'.format(MAXQUANT_TXT_DIR)
APL_DIR = '{}/andromeda'.format(BASE_MAXQUANT_DIR)

In [14]:
allpeptides_df = pd.read_csv(ALLPEPTIDES_FILENAME, sep='\t')
allpeptides_df.rename(columns={'Number of isotopic peaks':'isotope_count', 'm/z':'mz', 'Number of data points':'number_data_points', 'Intensity':'intensity', 'Ion mobility index':'scan', 'Ion mobility index length':'scan_length', 'Ion mobility index length (FWHM)':'scan_length_fwhm', 'Retention time':'rt', 'Retention length':'rt_length', 'Retention length (FWHM)':'rt_length_fwhm', 'Charge':'charge_state', 'Number of pasef MS/MS':'number_pasef_ms2_ids', 'Pasef MS/MS IDs':'pasef_msms_ids', 'MS/MS scan number':'msms_scan_number', 'Isotope correlation':'isotope_correlation'}, inplace=True)
# allpeptides_df = allpeptides_df[allpeptides_df.intensity.notnull() & (allpeptides_df.number_pasef_ms2_ids > 0) & (allpeptides_df.msms_scan_number >= 0) & allpeptides_df.pasef_msms_ids.notnull()].copy()
allpeptides_df = allpeptides_df[(allpeptides_df.msms_scan_number >= 0)].copy()
allpeptides_df.sort_values(by=['msms_scan_number'], ascending=True, inplace=True)

In [15]:
allpeptides_df.msms_scan_number = allpeptides_df.msms_scan_number.apply(lambda x: int(x))

In [16]:
allpeptides_df.msms_scan_number.max()

29928

In [26]:
allpeptides_df[(allpeptides_df['Raw file'] == 'YHE114_01_Slot1-1_1_3008') & (allpeptides_df.charge_state == 2)].head()

,Raw file,charge_state,mz,Mass,number_data_points,Number of frames,isotope_count,isotope_correlation,Mass fractional part,Mass deficit,...,Min frame index,Max frame index,scan,scan_length,scan_length_fwhm,intensity,Intensities,number_pasef_ms2_ids,pasef_msms_ids,msms_scan_number
103597,YHE114_01_Slot1-1_1_3008,2,356.19120,710.36784,4973,31,2,0.978443,0.367840,0.001071,...,1535,1565,855,24,15,72975.0,NaN,2,7483;7486,200
103688,YHE114_01_Slot1-1_1_3008,2,356.19121,710.36786,31608,105,2,0.987466,0.367858,0.001089,...,1382,1486,852,27,18,348470.0,NaN,11,3842;4105;4107;4151;4211;4612;4613;4934;4936;5...,201
103713,YHE114_01_Slot1-1_1_3008,2,356.19135,710.36816,3956,22,2,0.993326,0.368157,0.001387,...,1135,1187,858,24,15,103620.0,NaN,2,110;111,202
103712,YHE114_01_Slot1-1_1_3008,2,356.19145,710.36834,3495,30,2,0.995587,0.368338,0.001568,...,1186,1220,858,21,15,87628.0,NaN,1,266,203
102775,YHE114_01_Slot1-1_1_3008,2,358.24160,714.46866,134,4,2,0.994498,0.468656,0.100001,...,3489,3493,819,21,15,8094.4,NaN,3,101594;101595;101596,207


In [24]:
allpeptides_df.columns

Index(['Raw file', 'charge_state', 'mz', 'Mass', 'number_data_points',
       'Number of frames', 'isotope_count', 'isotope_correlation',
       'Mass fractional part', 'Mass deficit', 'rt', 'rt_length',
       'rt_length_fwhm', 'Min frame index', 'Max frame index', 'scan',
       'scan_length', 'scan_length_fwhm', 'intensity', 'Intensities',
       'number_pasef_ms2_ids', 'pasef_msms_ids', 'msms_scan_number'],
      dtype='object')

In [18]:
allpeptides_df['Raw file'].unique()

array(['YHE114_01_Slot1-1_1_3008', 'YHE211_01_Slot1-1_1_2997'],
      dtype=object)

In [19]:
def collate_spectra_for_feature(ms1_d, ms2_df):
    # append the monoisotopic and the ms2 fragments to the list for MGF creation
    pairs_df = ms2_df[['mz', 'intensity']].copy().sort_values(by=['mz'], ascending=True)
    spectrum = {}
    spectrum["m/z array"] = pairs_df.mz.values
    spectrum["intensity array"] = pairs_df.intensity.values
    params = {}
    params["TITLE"] = "RawFile: {} Index: {} Charge: {} FeatureIntensity: {} RtApex: {}".format(ms1_d['raw_file'], ms1_d['mq_index'], ms1_d['charge'], ms1_d['intensity'], round(ms1_d['rt_apex'],2))
    params["INSTRUMENT"] = "ESI-QUAD-TOF"
    params["PEPMASS"] = "{} {}".format(round(ms1_d['monoisotopic_mz'],6), ms1_d['intensity'])
    params["CHARGE"] = "{}+".format(ms1_d['charge'])
    params["RTINSECONDS"] = "{}".format(round(ms1_d['rt_apex'],2))
    params["SCANS"] = "{}".format(int(ms1_d['rt_apex']))
    spectrum["params"] = params
    return spectrum

In [20]:
# build a list of indexes from the APL files
ms2_peaks = []
apl_indexes = []
for file in glob.glob("{}/*.apl".format(APL_DIR)):
    with open(file, 'r') as f:
        for line in f:
            line = line.rstrip()
            if len(line) > 0:
                if line.startswith("header="):
                    line_a = line.split(' ')
                    mq_index = int(line_a[3])
                    raw_file = line_a[1]
                if line[0].isdigit():
                    line_a = line.split('\t')
                    mz = float(line_a[0])
                    intensity = round(float(line_a[1]))
                    ms2_peaks.append((mz, intensity))
                if line.startswith("peaklist end"):
                    apl_indexes.append((mq_index, ms2_peaks.copy(), raw_file))
                    del ms2_peaks[:]
                    mq_index = 0
apl_indexes_df = pd.DataFrame(apl_indexes, columns=['mq_index','ms2_peaks','raw_file'])
apl_indexes_df.sort_values(by=['mq_index'], ascending=True, inplace=True)

In [21]:
for group_name,group_df in allpeptides_df.groupby('Raw file'):
    mgf_spectra = []
    mgf_file_name = '{}/{}-dwm.mgf'.format(BASE_MAXQUANT_DIR, group_name)
    file_apl_indexes_df = apl_indexes_df[(apl_indexes_df['raw_file'] == group_name)]
    for idx,row in group_df.iterrows():
        mq_index = row.msms_scan_number
        ms1_d = {'monoisotopic_mass':row.Mass, 
                 'charge':row.charge_state, 
                 'monoisotopic_mz':row.mz, 
                 'intensity':int(row.intensity), 
                 'scan_apex':row.scan, 
                 'rt_apex':row.rt,
                 'raw_file':row['Raw file'],
                 'mq_index':mq_index}
        df = file_apl_indexes_df[(file_apl_indexes_df.mq_index == mq_index)]
        ms2_peaks_df = pd.DataFrame(df.iloc[0].ms2_peaks, columns=['mz','intensity'])
        feature_spectra = collate_spectra_for_feature(ms1_d, ms2_peaks_df)
        mgf_spectra.append(feature_spectra)
    # generate the MGF for all the features
    print("generating MGF: {}".format(mgf_file_name))
    if os.path.isfile(mgf_file_name):
        os.remove(mgf_file_name)
    f = mgf.write(output=mgf_file_name, spectra=mgf_spectra)


generating MGF: /Users/darylwilding-mcbride/Downloads/experiments/YHE-R/YHE114_01_YHE211_01-combined/YHE114_01_Slot1-1_1_3008-dwm.mgf
generating MGF: /Users/darylwilding-mcbride/Downloads/experiments/YHE-R/YHE114_01_YHE211_01-combined/YHE211_01_Slot1-1_1_2997-dwm.mgf
